In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
# Create a spark session (which will run spark jobs)
spark = (
    SparkSession.builder.appName("MAST30034 Tutorial 1")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .getOrCreate()
)

22/09/09 16:54:26 WARN Utils: Your hostname, hexiangyideMacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 10.13.178.156 instead (on interface en0)
22/09/09 16:54:26 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/09 16:54:26 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
census_04A_age = spark.read.parquet('../data/curated/census_data_2021_04A.parquet')
census_04B_age = spark.read.parquet('../data/curated/census_data_2021_04B.parquet')
census_17A_income = spark.read.parquet('../data/curated/census_data_2021_17A.parquet')
census_17B_income = spark.read.parquet('../data/curated/census_data_2021_17B.parquet')
census_17C_income = spark.read.parquet('../data/curated/census_data_2021_17C.parquet')
correspondences = spark.read.parquet('../data/curated/correspondences.parquet')
postcode = spark.read.parquet('../data/curated/postcode_SA2.parquet/')


In [3]:
#Merge the age dataframe
Age_Data = census_04A_age.join(census_04B_age).drop(census_04B_age.SA2_CODE_2021)

In [4]:
#Select the column in the age dataframe (18-65)
Age_Data_select = Age_Data.select(Age_Data.columns[0::18])
#Age_Data_select = Age_Data_select.select(Age_Data_select.columns[5:14])


In [5]:
# add all column together
Age_Data_select

SA2_CODE_2021,Age_yr_0_4_P,Age_yr_5_9_P,Age_yr_10_14_P,Age_yr_15_19_P,Age_yr_20_24_P,Age_yr_25_29_P,Age_yr_30_34_P,Age_yr_35_39_P,Age_yr_40_44_P,Age_yr_45_49_P,Age_yr_50_54_P,Age_yr_55_59_P,Age_yr_60_64_P,Age_yr_65_69_P,Age_yr_70_74_P,Age_yr_75_79_P,Tot_P
101021007,207,254,229,173,122,172,192,246,202,291,351,353,435,338,302,249,4343
101021007,207,254,229,173,122,172,192,246,202,291,351,580,513,429,325,227,8517
101021007,207,254,229,173,122,172,192,246,202,291,351,674,587,509,418,286,11342
101021007,207,254,229,173,122,172,192,246,202,291,351,323,296,216,168,126,5085
101021007,207,254,229,173,122,172,192,246,202,291,351,1006,663,452,366,196,12744
101021007,207,254,229,173,122,172,192,246,202,291,351,182,108,100,63,33,6224
101021007,207,254,229,173,122,172,192,246,202,291,351,1350,1206,947,804,441,16539
101021007,207,254,229,173,122,172,192,246,202,291,351,175,215,207,178,122,2454
101021007,207,254,229,173,122,172,192,246,202,291,351,469,478,365,399,308,6761
101021007,207,254,229,173,122,172,192,246,202,291,351,307,327,264,212,170,3472


In [6]:
from functools import reduce
from operator import add
from pyspark.sql.functions import col

Age_Data_total = Age_Data_select.withColumn("result", sum([col(x) for x in Age_Data_select.columns[5:14]]))

In [7]:
Age_Data_total

SA2_CODE_2021,Age_yr_0_4_P,Age_yr_5_9_P,Age_yr_10_14_P,Age_yr_15_19_P,Age_yr_20_24_P,Age_yr_25_29_P,Age_yr_30_34_P,Age_yr_35_39_P,Age_yr_40_44_P,Age_yr_45_49_P,Age_yr_50_54_P,Age_yr_55_59_P,Age_yr_60_64_P,Age_yr_65_69_P,Age_yr_70_74_P,Age_yr_75_79_P,Tot_P,result
101021007,207,254,229,173,122,172,192,246,202,291,351,353,435,338,302,249,4343,2364.0
101021007,207,254,229,173,122,172,192,246,202,291,351,580,513,429,325,227,8517,2669.0
101021007,207,254,229,173,122,172,192,246,202,291,351,674,587,509,418,286,11342,2837.0
101021007,207,254,229,173,122,172,192,246,202,291,351,323,296,216,168,126,5085,2195.0
101021007,207,254,229,173,122,172,192,246,202,291,351,1006,663,452,366,196,12744,3245.0
101021007,207,254,229,173,122,172,192,246,202,291,351,182,108,100,63,33,6224,1866.0
101021007,207,254,229,173,122,172,192,246,202,291,351,1350,1206,947,804,441,16539,4132.0
101021007,207,254,229,173,122,172,192,246,202,291,351,175,215,207,178,122,2454,1966.0
101021007,207,254,229,173,122,172,192,246,202,291,351,469,478,365,399,308,6761,2523.0
101021007,207,254,229,173,122,172,192,246,202,291,351,307,327,264,212,170,3472,2210.0


In [8]:
Age_Data_total = Age_Data_total.select("SA2_CODE_2021","result","Tot_P")
Age_Data_total

SA2_CODE_2021,result,Tot_P
101021007,2364.0,4343
101021007,2669.0,8517
101021007,2837.0,11342
101021007,2195.0,5085
101021007,3245.0,12744
101021007,1866.0,6224
101021007,4132.0,16539
101021007,1966.0,2454
101021007,2523.0,6761
101021007,2210.0,3472


In [10]:
Age_Data_rate = Age_Data_total.withColumn("percentage", Age_Data_total.result /Age_Data_total.Tot_P)
Age_Data_rate = Age_Data_rate.select("SA2_CODE_2021","percentage")

In [11]:
Age_Data_rate

SA2_CODE_2021,percentage
101021007,0.5443241998618467
101021007,0.313373253493014
101021007,0.25013225180744136
101021007,0.43166175024582104
101021007,0.25462962962962965
101021007,0.29980719794344474
101021007,0.24983372634379344
101021007,0.8011409942950285
101021007,0.37316964946013903
101021007,0.636520737327189
